In [1]:
# mlp : 다층 퍼셉트론
# cifar10 데이터셋 처리
import numpy as np
from tensorflow.keras import datasets, utils

In [2]:
(x_train,y_train),(x_test,y_test) = datasets.cifar10.load_data()
NUM_CLASSES = 10
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

y_train = utils.to_categorical(y_train,NUM_CLASSES)
y_test = utils.to_categorical(y_test,NUM_CLASSES)

170498071/170498071 [==============================] - 3s 0us/step


In [3]:
x_train.shape, x_train[54,12,13,1]  # 54인덱스의 12,13위치에 있는 녹색의 채널 값

((50000, 32, 32, 3), 0.36862746)

In [4]:
# Sequential 한 모델
from tensorflow.keras import layers,models
model = models.Sequential([
    layers.Flatten(input_shape = x_train[0].shape ),  # 32 32 3
    layers.Dense(200,activation='relu'),
    layers.Dense(150,activation='relu'),
    layers.Dense(NUM_CLASSES,activation='softmax'),
])

In [5]:
# 함수형 API를 이용한 MLP
input_layer = layers.Input(shape = x_train[0].shape)
x = layers.Flatten()(input_layer)
x = layers.Dense(200,activation='relu')(x)
x = layers.Dense(150,activation='relu')(x)
output_layer = layers.Dense(10,activation='softmax')(x)
model = models.Model(input_layer,output_layer)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_3 (Dense)             (None, 200)               614600    
                                                                 
 dense_4 (Dense)             (None, 150)               30150     
                                                                 
 dense_5 (Dense)             (None, 10)                1510      
                                                                 
Total params: 646260 (2.47 MB)
Trainable params: 646260 (2.47 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# 모델 컴파일
import tensorflow as tf
from tensorflow.keras import optimizers
opt = optimizers.Adam(learning_rate = 0.0005)
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])

In [8]:
# 모델 훈련
model.fit(x_train,y_train,batch_size=32,epochs=10,shuffle = True)

Epoch 1/10
1563/1563 [==============================] - 11s 4ms/step - loss: 1.8415 - accuracy: 0.3357
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.6719 - accuracy: 0.4042
Epoch 3/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5936 - accuracy: 0.4324
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.5465 - accuracy: 0.4467
Epoch 5/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5013 - accuracy: 0.4661
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.4695 - accuracy: 0.4766
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.4440 - accuracy: 0.4847
Epoch 8/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.4108 - accuracy: 0.4984
Epoch 9/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.3889 - accuracy: 0.5051
Epoch 10/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.36

In [ ]:
preds = model.predict(x_test)
preds

In [15]:
preds_index =  np.argmax( preds, axis = -1)
actual_index = np.argmax( y_test, axis = -1)
preds_index,actual_index

(array([5, 9, 0, ..., 3, 4, 7]), array([3, 8, 8, ..., 5, 1, 7]))

In [18]:
sum(preds_index == actual_index)/len(actual_index)

0.4826

In [25]:
# cnn 모델을 생성
# padding을 same이라고 하면 층을 통과한 텐서의 크기를 파악하기 쉬움
# (입력높이 / 스트라이드, 입력너비/스트라이드, 필터개수)

input_layer = layers.Input(shape = x_train[0].shape)  # 32 32 3
con1 = layers.Conv2D(10,kernel_size=(4,4),strides=2,padding='same')(input_layer)  # (none, 16,16,10)
con2 = layers.Conv2D(20,kernel_size=(3,3),strides=2,padding='same')(con1)  # (none, 8,8,20)
flatten_layer = layers.Flatten()(con2) # 8*8*20   1280
x = layers.Dense(200,activation='relu')(flatten_layer)
x = layers.Dense(150,activation='relu')(x)
output_layer = layers.Dense(NUM_CLASSES,activation='softmax')(x)
model = models.Model(input_layer, output_layer)

In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 10)        490       
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 20)          1820      
                                                                 
 flatten_3 (Flatten)         (None, 1280)              0         
                                                                 
 dense_7 (Dense)             (None, 200)               256200    
                                                                 
 dense_8 (Dense)             (None, 150)               30150     
                                                                 
 dense_9 (Dense)             (None, 10)                1510

In [27]:
opt = optimizers.Adam(learning_rate = 0.0005)  # 옵티마이져는 한번 학습되면 재 사용 안됨
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=10,shuffle = True)

Epoch 1/10
1563/1563 [==============================] - 8s 4ms/step - loss: 1.6670 - accuracy: 0.4042
Epoch 2/10
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3833 - accuracy: 0.5109
Epoch 3/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.2501 - accuracy: 0.5574
Epoch 4/10
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1548 - accuracy: 0.5909
Epoch 5/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0675 - accuracy: 0.6210
Epoch 6/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9828 - accuracy: 0.6527
Epoch 7/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8994 - accuracy: 0.6818
Epoch 8/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8132 - accuracy: 0.7128
Epoch 9/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.7348 - accuracy: 0.7411
Epoch 10/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.652

In [29]:
# 학습을 원할하게 하고 과적합을 회피하는 방법
# 배치정규화, 드랍아웃
# 컨볼류션, 배치정규화, 드랍아웃 CNN
input_layer = layers.Input(shape = x_train[0].shape)  # 32 32 3
x = layers.Conv2D(32,kernel_size=(4,4),strides=1,padding='same')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(32,kernel_size=(4,4),strides=2,padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64,kernel_size=(4,4),strides=1,padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64,kernel_size=(4,4),strides=2,padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

x = layers.Dense(200,activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(rate=0.5)(x)

output_layer = layers.Dense(NUM_CLASSES,activation='softmax')(x)

model = models.Model(input_layer, output_layer)

In [ ]:
opt = optimizers.Adam(learning_rate = 0.0005)  # 옵티마이져는 한번 학습되면 재 사용 안됨
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=10,shuffle = True)

Epoch 1/10
